# Neural Network Smartphone Activity Detector

In this activity, you will train a neural network to use smartphone data to predict the activity of the user. 

This dataset has already been separated into input features and target activities. Additional information on the dataset can be found here. 

http://archive.ics.uci.edu/ml/datasets/Smartphone-Based+Recognition+of+Human+Activities+and+Postural+Transitions

### Data Pre-Processing

Prepare the data for the neural network. This includes splitting the data into a training and testing dataset, Scaling the data, and encoding the categorical target values

In [162]:
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [163]:
# Read the input features into `X`
X = pd.read_csv(Path("../Resources/features.csv"), header=None)
X.head()

0         1         2         3         4         5         6    \
0  0.043580 -0.005970 -0.035054 -0.995381 -0.988366 -0.937382 -0.995007   
1  0.039480 -0.002131 -0.029067 -0.998348 -0.982945 -0.971273 -0.998702   
2  0.039978 -0.005153 -0.022651 -0.995482 -0.977314 -0.984760 -0.996415   
3  0.039785 -0.011809 -0.028916 -0.996194 -0.988569 -0.993256 -0.996994   
4  0.038758 -0.002289 -0.023863 -0.998241 -0.986774 -0.993115 -0.998216   

        7         8         9    ...       551       552       553       554  \
0 -0.988816 -0.953325 -0.794796  ... -0.012236 -0.314848 -0.713308 -0.112754   
1 -0.983315 -0.974000 -0.802537  ...  0.202804 -0.603199 -0.860677  0.053477   
2 -0.975835 -0.985973 -0.798477  ...  0.440079 -0.404427 -0.761847 -0.118559   
3 -0.988526 -0.993135 -0.798477  ...  0.430891 -0.138373 -0.491604 -0.036788   
4 -0.986479 -0.993825 -0.801982  ...  0.137735 -0.366214 -0.702490  0.123320   

        555       556       557       558       559       560  
0  0.030400 -0.464761 -0.018446 -0.841559  0.179913 -0.051718  
1 -0.007435 -0.732626  0.703511 -0.845092  0.180261 -0.047436  
2  0.177899  0.100699  0.808529 -0.849230  0.180610 -0.042271  
3 -0.012892  0.640011 -0.485366 -0.848947  0.181907 -0.040826  
4  0.122542  0.693578 -0.615971 -0.848164  0.185124 -0.037080  

[5 rows x 561 columns]

In [164]:
# Read the target values into `y`
y = pd.read_csv(Path("../Resources/target.csv"))
y.head()

activity
0  standing
1  standing
2  standing
3  standing
4  standing

In [165]:
y.activity.value_counts()

standing              1423
laying                1413
sitting               1293
walking               1226
walking_upstairs      1073
walking_downstairs     987
stand_to_lie            90
sit_to_lie              75
lie_to_sit              60
lie_to_stand            57
stand_to_sit            47
sit_to_stand            23
Name: activity, dtype: int64

In [166]:
# Split the dataset into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [167]:
# Scale the training and testing input features using StandardScaler
X_scaler = StandardScaler()
X_scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [168]:
# Apply One-hot encoding to the target labels
enc = OneHotEncoder()
enc.fit(y_train)

encoded_y_train = enc.transform(y_train).toarray()
encoded_y_test = enc.transform(y_test).toarray()

# class_values = y["activity"].values.reshape(-1, 1)
# enc.fit(class_values)
# enc.categories_


# Build a Deep Neural Network

In [169]:
# Create a sequential model
neuron = Sequential()


In [170]:
# Add the first layer where the input dimensions are the 561 columns of the training data
neuron.add(Dense(100, activation='relu', input_dim=X_train_scaled.shape[1]))
# number_hidden_nodes = 1500
#neuron.add(Dense(100, activation="relu", input_dim=number_inputs))
# model.add(Dense(100, activation='relu', input_dim=X_train_scaled.shape[1]))


In [171]:
# The output layer has 12 columns that are one-hot encoded
y_train.activity.value_counts()
number_outputs = 12

In [172]:
# Add output layer using 12 output nodes. 
# HINT: Use `softmax` as the activation 
# number_classes = 561
# neuron.add(Dense(units=number_classes, activation="softmax"))
neuron.add(Dense(number_outputs, activation="softmax"))


In [173]:
# Compile the model using categorical_crossentropy for the loss function, the adam optimizer,
# and add accuracy to the training metrics
neuron.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [174]:
# Print the model summary
neuron.summary()


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 100)               56200     
_________________________________________________________________
dense_22 (Dense)             (None, 12)                1212      
Total params: 57,412
Trainable params: 57,412
Non-trainable params: 0
_________________________________________________________________


In [175]:
# Use the training data to fit (train) the model
# @NOTE: Experiment with the number of training epochs to find the minimum iterations required to achieve a good accuracy
# model = neuron.fit(X_train_scaled, y_train, epochs=100)

from tensorflow.keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=2)

neuron.fit(
    X_train_scaled,
    encoded_y_train,
    epochs=100,
    shuffle=True,
    verbose=2
    
#     callbacks= [early_stopping_monitor]
)


Epoch 1/100
183/183 - 0s - loss: 0.0527 - accuracy: 0.8721
Epoch 2/100
183/183 - 0s - loss: 0.0210 - accuracy: 0.9506
Epoch 3/100
183/183 - 0s - loss: 0.0145 - accuracy: 0.9679
Epoch 4/100
183/183 - 0s - loss: 0.0135 - accuracy: 0.9712
Epoch 5/100
183/183 - 0s - loss: 0.0088 - accuracy: 0.9801
Epoch 6/100
183/183 - 0s - loss: 0.0075 - accuracy: 0.9840
Epoch 7/100
183/183 - 0s - loss: 0.0058 - accuracy: 0.9892
Epoch 8/100
183/183 - 0s - loss: 0.0067 - accuracy: 0.9854
Epoch 9/100
183/183 - 0s - loss: 0.0057 - accuracy: 0.9885
Epoch 10/100
183/183 - 0s - loss: 0.0044 - accuracy: 0.9911
Epoch 11/100
183/183 - 0s - loss: 0.0033 - accuracy: 0.9938
Epoch 12/100
183/183 - 0s - loss: 0.0036 - accuracy: 0.9935
Epoch 13/100
183/183 - 0s - loss: 0.0031 - accuracy: 0.9938
Epoch 14/100
183/183 - 0s - loss: 0.0031 - accuracy: 0.9936
Epoch 15/100
183/183 - 0s - loss: 0.0025 - accuracy: 0.9936
Epoch 16/100
183/183 - 0s - loss: 0.0017 - accuracy: 0.9967
Epoch 17/100
183/183 - 0s - loss: 0.0015 - accura

# Evaluate the Model

In [180]:
# Evaluate the model using the testing data
model_loss, model_accuracy = neuron.evaluate(X_test_scaled, encoded_y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# model_loss, model_accuracy = neuron.evaluate(X_test_scaled, encoded_y_test, verbose=2)
# print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")


61/61 - 0s - loss: 0.0267 - accuracy: 0.9706
Loss: 0.026719270274043083, Accuracy: 0.9706488251686096


In [181]:
# Make predictions
predicted = neuron.predict(X_test_scaled)
predicted = enc.inverse_transform(predicted).flatten().tolist()
results = pd.DataFrame({
    "Actual": y_test.activity.values,
    "Predicted": predicted
})
results.head(10)

Actual           Predicted
0    walking_upstairs    walking_upstairs
1             walking             walking
2  walking_downstairs  walking_downstairs
3              laying              laying
4    walking_upstairs    walking_upstairs
5             sitting             sitting
6            standing            standing
7  walking_downstairs  walking_downstairs
8  walking_downstairs  walking_downstairs
9  walking_downstairs  walking_downstairs

In [182]:
# Print the Classification Report
from sklearn.metrics import classification_report
print(classification_report(results.Actual, results.Predicted))

                    precision    recall  f1-score   support

            laying       1.00      0.99      1.00       331
        lie_to_sit       0.74      0.81      0.77        21
      lie_to_stand       0.69      0.60      0.64        15
        sit_to_lie       0.65      0.72      0.68        18
      sit_to_stand       0.83      1.00      0.91         5
           sitting       0.96      0.97      0.97       355
      stand_to_lie       0.68      0.65      0.67        20
      stand_to_sit       1.00      0.89      0.94        18
          standing       0.97      0.96      0.96       361
           walking       0.99      0.99      0.99       305
walking_downstairs       0.99      1.00      0.99       244
  walking_upstairs       0.99      0.99      0.99       249

          accuracy                           0.97      1942
         macro avg       0.87      0.88      0.88      1942
      weighted avg       0.97      0.97      0.97      1942



In [179]:
# Create a DataFrame with the history dictionary
df = pd.DataFrame(neuron.history, index=range(1, len(neuron.history["loss"]) + 1))

# # Plot the loss
# df.plot(y="loss")


TypeError: 'History' object is not subscriptable